In [1]:
## basic library requirements
from __future__ import division

import os
import urllib, cStringIO

import numpy as np
import scipy.stats as stats
import pandas as pd
import json
import re

from PIL import Image
import base64
import datetime
import time

# set path to database connectinos 
auth = pd.read_csv('../auth.txt', header = None) 
pswd = auth.values[0][0]

## use pymongo for database
import pymongo as pm
conn = pm.MongoClient('mongodb://stanford:' + pswd + '@127.0.0.1')
db = conn['kiddraw']
cdm_run_v8 = db['cdm_run_v8']
cdm_run_v7 = db['cdm_run_v7']
cdm_run_v6 = db['cdm_run_v6']
cdm_run_v5 = db['cdm_run_v5']
cdm_run_v4 = db['cdm_run_v4']
cdm_run_v3 = db['cdm_run_v3']

###### ###### ###### TOGGLE HERE WHICH DATABSE
this_collection = cdm_run_v3
which_run = 'cdm_run_v3'
###### ###### ###### ######

In [2]:

###### Where are we saving
analysis_dir = os.getcwd()

output_dir = os.path.join(analysis_dir,'full_dataset_info')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [3]:
###### Define functions for use below in getting img intensities and bounding boxes
def load_image_data(imgData,imsize):
    filestr = base64.b64decode(imgData)
    fname = os.path.join('sketch.png')
    with open(fname, "wb") as fh:
        fh.write(imgData.decode('base64'))
    im = Image.open(fname).resize((imsize,imsize))
    _im = np.array(im)
    return(_im)

def get_mean_intensity(img,imsize):
    thresh = 250
    numpix = imsize**2
    mean_intensity = len(np.where(img[:,:,3].flatten()>thresh)[0])/numpix
    return mean_intensity
    
def get_bounding_box(img):
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]
    bounding_box = tuple((rmin, rmax, cmin, cmax))
    return bounding_box

In [4]:
######MASSIVE LOOP TO RENDER OUT IMAGES & INFO
# basic variables for counting throughout the loop
skipCount = 0;
writeImageCount = 0;
interferenceCount = 0;
timeSave = []
imsize = 224 ## img writing size, optimized for CNN
countImage = 0

## Get all sessions within this collection
sessions_this_coll =  this_collection.find({'$and': [{'dataType':'finalImage'}]}).distinct('sessionId')
time_start = time.time() ## 

In [5]:
# post filtering
valid_sessions = pd.read_csv('valid_sessions.csv')
valid_sessions.columns= ['session_id']

sessions_this_coll = pd.DataFrame(sessions_this_coll)
sessions_this_coll.columns = ['session_id']

In [6]:
sessions_to_render = pd.merge(sessions_this_coll, valid_sessions, how ='inner', on =['session_id']) 
print('we have {} unique kids'.format(len(sessions_to_render)))


we have 1937 unique kids


In [7]:
categories =  this_collection.find({'$and': [{'dataType':'finalImage'}]}).distinct('category')
ages =  this_collection.find({'$and': [{'dataType':'finalImage'}]}).distinct('age')


In [14]:
imrec

{u'_id': ObjectId('5af33457f1d6e2624479c1ad'),
 u'age': u'age6',
 u'category': u'a cup',
 u'dataType': u'finalImage',
 u'date': u'2018-05-09T17:48:07.740Z',
 u'imgData': u'iVBORw0KGgoAAAANSUhEUgAAAJYAAACWCAYAAAA8AXHiAAAAAXNSR0IArs4c6QAAB51JREFUeAHtnTvIHUUUxz+fqIiNrxSCTUglEWxUMJ0KxkJbsRLEIoKgnYVgYaOVop3a2MdCBIsI2qmNShSLIIgoEkNIoRIliY9zcP8fm707e3fnzuOcnf/COnd3Zs7jd/7Z3bvf3nVvjwsJkAAJkAAJkAAJkAAJkAAJkAAJkAAJkAAJkAAJkAAJkAAJkAAJkAAJkEAuAv+KYV2bWa5sJtN6iTYlKGC+Ah/YZiHQF1VTrJtKNot0xo32BaUjmuPMU+G4MGL2HpVJKqjmRRUDj3M2CbzRiQmigrCwvTmjgT1XN5BjrhQhoL59nPLQd0O/k59JIETgd+nAkajfDsejb7if2ySwT+Ar+QSh9NuD+yM2P2DcZg/3NEngJsn6XVkhjGH7+QwqZ7r5h2aM5ZAVEbivK/xQNKHtTxfmDjsLp3G4BwLPSpAo8NL2rR0ThL8dzXB6TQL6reucrCjmnPbhzAFrDJcy+6D5SALXyLzbZJ0jlLExRyL97jrt0S7mE7sa4vzdCTzQFWNMINv2fbe7+6QWEG9SozQWRwDF6LcfiKljnTnsj7NedpanWLOSsXLnXQvCv1tmLXVZ4xaEhT+DlM2c3rIS4FEiK952jXsQ1umuPHe1WyZmnouAl4tiL3HmqtO+XQ9HrP1g+cEPAQrLT61cRUphuSqXn2C9CEuvXbg4IuBFWPpnH13e/7/hf0kgHQEP37g8xJiuIhOWvByxJlJgl0UCFJbFqqwgJgprBUW0mAKFZbEqK4iJwlpB

In [15]:
x

session cdm_run_v31525887854758
session cdm_run_v31525890266406
session cdm_run_v31525891444648
session cdm_run_v31525891955621
session cdm_run_v31525893174061
session cdm_run_v31525893423612
session cdm_run_v31525893976840
session cdm_run_v31525895728803
session cdm_run_v31525899254114
session cdm_run_v31525901322190
session cdm_run_v31525901714819
session cdm_run_v31525904067360
session cdm_run_v31525908206117
session cdm_run_v31525908387543
session cdm_run_v31525908911986
session cdm_run_v31525974358965
session cdm_run_v31525975379407
session cdm_run_v31525976032935
session cdm_run_v31525976865876
session cdm_run_v31525978723767
session cdm_run_v31525984464310
session cdm_run_v31525990258467
session cdm_run_v31525992411710
session cdm_run_v31525992682386
session cdm_run_v31525993726883
session cdm_run_v31526060310716
session cdm_run_v31526060708396
session cdm_run_v31526060998610
session cdm_run_v31526061437727
session cdm_run_v31526061799166
session cdm_run_v31526061922561
session 

session cdm_run_v31528064619442
session cdm_run_v31528064999186
session cdm_run_v31528065411124
session cdm_run_v31528065606112
session cdm_run_v31528065690229
session cdm_run_v31528065927196
session cdm_run_v31528066914558
session cdm_run_v31528067999106
session cdm_run_v31528068300688
session cdm_run_v31528068401956
session cdm_run_v31528135799225
session cdm_run_v31528136180481
session cdm_run_v31528136399273
session cdm_run_v31528136881460
session cdm_run_v31528136999209
session cdm_run_v31528137160060
session cdm_run_v31528137433578
session cdm_run_v31528137599244
session cdm_run_v31528139252267
session cdm_run_v31528140599196
session cdm_run_v31528148156599
session cdm_run_v31528152599102
session cdm_run_v31528154999081
session cdm_run_v31528222799238
session cdm_run_v31528223120387
session cdm_run_v31528224184593
session cdm_run_v31528224495202
session cdm_run_v31528224599151
session cdm_run_v31528225564149
session cdm_run_v31528227893962
session cdm_run_v31528228199254
session 

KeyboardInterrupt: 

In [19]:
np.size(sessions_to_render)

1937

In [20]:
    time_start = time.time() ## 


In [21]:
time_start


1616181663.879985

In [28]:
count_session=10
if (count_session%10)==0):
    print('test')

NameError: name 'mod' is not defined